# Lecture1.introduction
图像被称作互联网的“暗物质”  
David Marr，1970s，stages of Visual Representation  
    1.input image
    2.primal sketch(edge image)
    3.2 1/2-D sketch 
    4.3-D model
Face Detection,2001  
Histogam of Gradients(HoG),Dalal & Triggs,2005  
PASCAL Visual Object Challenge(20 object categories)  
_现代图像识别问题是特征多，维度高，用算法经常过拟合_  

*Image Net Challenge所用算法演变*  
Lin CVPR 2011---svm  
Krizhevsky NIPS 2012------cNN,Supervision(AlexNet)  
Szegedy arxiv 2014/Simonyan arxiv 2014-------VGG GoogleNet  
Microsoft Research Asia 2015-----152 layer Residual Networks

# Lecture2.image classificatioon
### Data-Driven Approach
1. collect a dataset of images and labels
2. use machine learning to train a classifier
3. evaluate the classifier on new images  

### 机器学习方法用于预测分类，，一般分两个函数（步骤）：
1. 输入函数→train
2. 输出函数→predict  

### 常用分类器1：Nearest Neighbor
1. memorize all data and labels
2. predict the label of the most similar train image  

*Distance Metric to compare images*  
L1 distance(Manhattan distance):$d_1(I_1,I_2)=\sum_{P}\left | I_1^P - I_2^P \right |$  

训练时间复杂度O(1), 预测时间复杂度O(N)，但是我们的需求是，训练时间可以长，但是预测速度越快越好  
_缺点_：不准确,噪声点误分类  
_改进_：K-Nearest Neighbor，给定一个K，将最邻近的K个样本点的分类作为最终预测结果

*用欧几里得距离作为Distance Meric*  
L2(Euclidean)distance:$d_2(I_1,I_2)= \sqrt{\sum_{P}(I_1^P - I_2^P)^{2}}$  
#### L1与L2区别 
L1依赖于所选择的坐标系，若旋转坐标系，L1距离会变化  
L2不依赖于坐标系  
如果输入特征有特别含义，则用L1较好，如果特征间无差别，则用L2较好 

#### 设置K近邻的超参数  
1. ×选择最佳超参数K（BAD：K=1总是对训练集拟合最好）
2. ×分为训练集和测试集（BAD：只在测试集上预测效果好，不知道未知数据预测效果如何）  
3. √分为训练集、验证集和测试集
4. √cross-validation（在小数据集中非常有用，但在deep learning中不常用）

_K-Nearest Neighbor on images **never used**_
- Very slow at test time
- Distance metrics on pixels are not informative
L2距离对样本数据变化（图像变化，如遮挡，变换）不敏感
- curse of dimensionality
随着维度增加，数据个数（采样点）指数级增多  

#### summary
- In image classification we start with a training set of images and labels , andd must predict labels on the test set 
- The K-Neatest Neighbors classifier predicts labels based on nearest training examples
- Distance metric and K are hyperparameters
- Choose hyperparameters using the validation set; only run on the test set once at the very end!  

### 常用分类器2：线性分类
#### Parametric Approach
一个尺寸为32（pixels）×32（pixels）×3（RGB）的图片，转为含3072数字的Array，通过权重矩阵W，转换为10个给定分类的分值  
$f(x) = Wx + b$

*不需要测试集*

该方法试图在高维空间用线性划分分类,但对线性不可分集无用  

如何用cost function评价W的好坏，下节课讲

# Lecture3.Loss Functions and Optimization
